In [1]:
from pyspark.sql import SparkSession
import psycopg2
import pandas as pd
import os
from pyspark.sql import SparkSession
from sqlalchemy import create_engine
from pyspark import SparkConf

# initialise sparkContext

spark = SparkSession.builder.config("spark.jars", "postgresql-42.2.5.jar")\
    .config("spark.jars.packages","org.postgresql:postgresql:42.2.5")\
    .config('spark.driver.maxResultSize','20g').config("spark.executor.memory", "20g")\
    .config("spark.driver.memory","20g").config("spark.executor.memory","20g")\
    .config("spark.shuffle.file.buffer",'40').config("spark.scheduler.listenerbus.eventqueue.capacity","10000000")\
    .master("local").appName("tinkof-invest").getOrCreate()
spark.conf.set("spark.sql.execution.arrow.enabled", True)
spark.sparkContext.setLogLevel("ERROR")
spark


22/05/06 20:41:58 WARN Utils: Your hostname, nikolay resolves to a loopback address: 127.0.1.1; using 192.168.43.45 instead (on interface wlp2s0)
22/05/06 20:41:58 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/nikolay/anaconda3/lib/python3.7/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/nikolay/.ivy2/cache
The jars for the packages stored in: /home/nikolay/.ivy2/jars
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-92419c49-88d6-4268-88d5-8436a6045672;1.0
	confs: [default]
	found org.postgresql#postgresql;42.2.5 in central
:: resolution report :: resolve 216ms :: artifacts dl 9ms
	:: modules in use:
	org.postgresql#postgresql;42.2.5 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-92419c49-88d6-4268-88d5-8436a6045672
	confs:

In [16]:
shares_info = spark.createDataFrame(pd.read_parquet('Shares.parquet'))

In [17]:
shares_info

DataFrame[figi: string, ticker: string, class_code: string, isin: string, lot: bigint, currency: string, klong: decimal(9,8), kshort: decimal(9,8), dlong: decimal(9,8), dshort: decimal(9,8), dlong_min: decimal(9,8), dshort_min: decimal(9,8), short_enabled_flag: decimal(9,8), name: string, exchange: string, ipo_date: timestamp, issue_size: bigint, nominal: decimal(13,9), trading_status: bigint, otc_flag: boolean, buy_available_flag: boolean, sell_available_flag: boolean, div_yield_flag: boolean, share_type: bigint, min_price_increment: decimal(10,8), api_trade_available_flag: boolean, uid: string, real_exchange: bigint]

In [18]:
shares_info.write.format('jdbc').option("url","jdbc:postgresql://localhost:5432/test_db")\
    .option("table",'shares_info').option("user",'nikolay').option("password","Andone1")\
    .option("mode","overwrite")

In [19]:
shares_info.summary()

DataFrame[summary: string, figi: string, ticker: string, class_code: string, isin: string, lot: string, currency: string, klong: string, kshort: string, dlong: string, dshort: string, dlong_min: string, dshort_min: string, short_enabled_flag: string, name: string, exchange: string, issue_size: string, nominal: string, trading_status: string, share_type: string, min_price_increment: string, uid: string, real_exchange: string]

In [21]:
mode = "overwrite"
url = "jdbc:postgresql://localhost:5432/shares"
properties = {"user": "nikolay","password": "Andone1","driver": "org.postgresql.Driver"}
shares_info.write.jdbc(url=url, table="shares_info", mode=mode, properties=properties)

In [22]:
all_candles_day = spark.createDataFrame(pd.read_parquet('Shares_market_data2000_2022.parquet'))
# all_candles_day.write.format('jdbc').option("url","jdbc:postgresql://localhost:5432/test_db")\
#     .option("table",'candles_day').option("user",'nikolay').option("password","Andone1")\
#     .option("mode","overwrite")

In [23]:
all_candles_day.write.jdbc(url=url, table="candles_day", mode=mode, properties=properties)

In [25]:
t = pd.read_parquet('Shares.parquet')

In [26]:
t

,figi,ticker,class_code,isin,lot,currency,klong,kshort,dlong,dshort,...,trading_status,otc_flag,buy_available_flag,sell_available_flag,div_yield_flag,share_type,min_price_increment,api_trade_available_flag,uid,real_exchange
0,BBG000HLJ7M4,IDCC,SPBXM,US45867G1013,1,usd,0E-8,0E-8,0E-8,0E-8,...,5,False,True,True,True,1,0.01000000,True,,2
1,BBG002293PJ4,RH,SPBXM,US74967X1037,1,usd,0E-8,0E-8,0E-8,0E-8,...,5,False,True,True,False,1,0.01000000,True,,2
2,BBG000BH0FR6,SGEN,SPBXM,US81181C1045,1,usd,0E-8,0E-8,0E-8,0E-8,...,5,False,True,True,False,1,0.01000000,True,,2
3,BBG004MN1R41,NAVI,SPBXM,US63938C1080,1,usd,0E-8,0E-8,0E-8,0E-8,...,5,False,True,True,True,1,0.01000000,True,,2
4,BBG000BWS3F3,WGO,SPBXM,US9746371007,1,usd,0E-8,0E-8,0E-8,0E-8,...,5,False,True,True,True,1,0.01000000,True,,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1850,BBG00N6NL712,RIDE,SPBXM,US54405Q1004,1,usd,2.00000000,2.00000000,0.55110000,0.76890000,...,5,False,True,True,False,1,0.01000000,True,,2
1851,BBG002W2FT69,ABRD,TQBR,RU000A0JS5T7,10,rub,2.00000000,2.00000000,0.30000000,0.41920000,...,4,False,True,True,True,1,0.50000000,True,,1
1852,BBG000BWJFZ4,CF,SPBXM,US1252691001,1,usd,2.00000000,2.00000000,0.25000000,0.27690000,...,5,False,True,True,True,1,0.01000000,True,,2
1853,BBG005BT60Y8,NVEE,SPBXM,US62945V1098,1,usd,0E-8,0E-8,0E-8,0E-8,...,5,False,True,True,False,1,0.01000000,True,,2
